In [2]:
!pip install tensorflow

     ------------------------------------ 266.3/266.3 MB 932.0 kB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     -------------------------------------- 439.2/439.2 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 2.6 MB/s eta 0:00:00
  Using cached tensorflow_io_gcs_filesystem-0.28.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     ---------------------------------------- 1.7/1.7 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 1.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1


In [31]:
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import pathlib
import tensorflow as tf
import tensorflow as keras
from keras import Model, layers
from keras.layers import *
from fastai.vision.all import *
#from utils.getScreen import grab_screen
from PIL import Image 
from PIL.ImageDraw import Draw
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img

In [6]:
training_csv_file = 'data.csv'
training_image_dir = 'C:/Users/Joshua/Desktop/Data/osu-ai'

In [7]:
training_image_records = pd.read_csv(training_csv_file)
train_image_path = os.path.join(os.getcwd(), training_image_dir)

In [8]:
train_images = []
train_targets = []

In [32]:
for index, row in training_image_records.iterrows():

    (filename, x, y) = row

    train_image_fullpath = os.path.join(train_image_path, filename)
    train_img = tf.keras.utils.load_img(train_image_fullpath, target_size=(600, 800))
    train_img_arr = tf.keras.utils.img_to_array(train_img)

    train_images.append(train_img_arr)
    train_targets.append((x, y))

MemoryError: Unable to allocate 5.49 MiB for an array with shape (600, 800, 3) and data type float32

In [ ]:
train_images = np.array(train_images)
train_targets = np.array(train_targets)

In [ ]:
validation_images = np.array(validation_images)
validation_targets = np.array(validation_targets)

In [ ]:
width = 800
height = 600

In [ ]:
#create the common input layer
input_shape = (height, width, 3)
input_layer = tf.keras.layers.Input(input_shape)

In [33]:
#create the base layers
base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer)
base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
base_layers = layers.Flatten(name='bl_8')(base_layers)

AttributeError: module 'keras.layers' has no attribute 'experimental'

In [34]:
#create the localiser branch
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)

NameError: name 'base_layers' is not defined

In [35]:
model = tf.keras.Model(input_layer, outputs=[locator_branch])

NameError: name 'input_layer' is not defined

In [ ]:
losses = {"bb_head":tf.keras.losses.MSE}

In [ ]:
model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])

In [ ]:
trainTargets = {
    "bb_head": train_targets
}
validationTargets = {
    "bb_head": validation_targets
}

In [ ]:
history = model.fit(train_images, trainTargets,
             validation_data=(validation_images, validationTargets),
             batch_size=4,
             epochs=20,
             shuffle=True,
             verbose=1)

In [ ]:
for layer in model.layers:
    if layer.name.startswith('bl_'):
        layer.trainable = False
        
for layer in model.layers:
    if layer.name.startswith('bb_'):
        layer.trainable = False